<h3> Explore Toronto </h3>
Import the Dataset from the wikipedia page, with the following assumptions:
- Not assigned Borough are deleted
- Not assigned Neighborhoods have the same name of their Boroughs
- Data are grouped by Postalcode

Latitude and longitudes of each neighborhood are also addedd.

The 'TorontoCAP_Coord.csv' file contains already cleaned data

<b>Import necessary packages: </b>

In [244]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

<b>Explore the dataset</b>

In [245]:
df=pd.read_csv('TorontoCAP_Coord.csv', delimiter=';', header=0)
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [246]:
df.shape
print('Toronto has {} boroughs'.format(
        len(df['Borough'].unique())
    )
)

Toronto has 13 boroughs


<b>Create a map of Toronto with neighborhoods on top</b>

In [247]:
# create map of Toronto using latitude and longitude values

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
       [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [248]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<b>Connect to Foursquare API to explore Toronto </b>

In [249]:
CLIENT_ID = 'YHIHTB2ZFNRG0DMWPKR2QY4YVRM1GQPTC0DFFJ3NQLRCR3SS' 
CLIENT_SECRET = 'K3SAZTMZZVN0SRMAMQQJAOHDR2QX5CFFZ0G1CPPVTBKULZAW' 
VERSION = '20180605' 

I want to explore only the borough <i>North York</i>

In [288]:
place=df.loc[df['Borough']=='North York'].reset_index(drop=True)
place

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
8,M2K,North York,Bayview Village,43.786947,-79.385975
9,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763


Explore the neighborhoods of North York

In [289]:
place.loc[0,'Neighborhood']
place_latitude = scar.loc[0,'Latitude'] 
place_longitude = scar.loc[0,'Longitude']
place_name = scar.loc[0,'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(place_name, 
                                                              place_latitude, 
                                                             place_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


<b>Get from FourSquare the top 50 venues in this neighborhood of north York within a radius of 500 meters.</b>

In [290]:
LIMIT=50; radius=500;

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    place_latitude, 
    place_longitude, 
    radius, 
    LIMIT)
results= requests.get(url).json()

In [291]:
#Extract information of venues and store them in a Dataframe

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
   
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


Run the same code to all neighborhoods 

In [293]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
     # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues);      

place_venues = getNearbyVenues(names=place['Neighborhood'],
                                   latitudes=place['Latitude'],
                                   longitudes=place['Longitude']
                                  );


Parkwoods
Victoria Village
Lawrence Heights, Lawrence Manor
Don  Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Bayview Village
CFB Toronto, Downsview East
Silver Hills, York Mills
Downsview, North Park, Upwood Park
Humber Summit
Newtonbrook, Willowdale
Downsview Central
Bedford Park, Lawrence Manor East
Emery, Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [294]:
print('North York has {} boroughs'.format(
 len(place_venues['Neighborhood'].unique())
)
   )

North York has 20 boroughs


In [295]:
place_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [296]:
place_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
"Downsview, North Park, Upwood Park",5,5,5,5,5,5
"Emery, Humberlea",1,1,1,1,1,1


In [297]:
print('In Nort York there are {} uniques categories.'.format(len(place_venues['Venue Category'].unique())))

In Nort York there are 88 uniques categories.


<b>Analysis of each neighborhood </b>

In [299]:
# one hot encoding
place_onehot = pd.get_dummies(place_venues[['Venue Category']], prefix="", prefix_sep="")

place_onehot['Neighborhood'] = place_venues['Neighborhood'] 

fixed_columns = [place_onehot.columns[-1]] + list(place_onehot.columns[:-1])
place_onehot = place_onehot[fixed_columns]
place_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Bus Line,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Group by neighborhood and print the top 10 venues 

In [300]:
place_grouped = place_onehot.groupby('Neighborhood').mean().reset_index()
place_grouped
num_top_venues = 10

for hood in place_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = place_grouped[place_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1         Frozen Yogurt Shop  0.06
2             Sandwich Place  0.06
3  Middle Eastern Restaurant  0.06
4              Deli / Bodega  0.06
5       Fast Food Restaurant  0.06
6                   Pharmacy  0.06
7                Pizza Place  0.06
8        Fried Chicken Joint  0.06
9                Bridal Shop  0.06


----Bayview Village----
                       venue  freq
0        Japanese Restaurant  0.25
1         Chinese Restaurant  0.25
2                       Café  0.25
3                       Bank  0.25
4                  Juice Bar  0.00
5              Movie Theater  0.00
6         Miscellaneous Shop  0.00
7  Middle Eastern Restaurant  0.00
8   Mediterranean Restaurant  0.00
9                     Lounge  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1     Fast Food Restaurant  0.08
2    

Save also these results in a dataframe and display the top ten

In [301]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Bridal Shop,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Cupcake Shop,Japanese Restaurant,Juice Bar,Café,Liquor Store,Ice Cream Shop,Butcher
3,"CFB Toronto, Downsview East",Airport,Other Repair Shop,Park,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
4,Don Mills North,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Baseball Field,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cupcake Shop


<b> Cluster neighborhoods with k-means algorithm </b>

In [302]:
# set number of clusters
kclusters = 6
place_grouped_clustering = place_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(place_grouped_clustering)
 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [303]:
place_merged = place
place_merged = place_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

place_merged.dropna(inplace=True)
place_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Women's Store,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Fraternity House,Vietnamese Restaurant,Accessories Store
3,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Baseball Field,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cupcake Shop
4,M6B,North York,Glencairn,43.709577,-79.445073,3.0,Park,Japanese Restaurant,Pizza Place,Pub,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
5,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1.0,Coffee Shop,Gym,Asian Restaurant,Beer Store,Clothing Store,Chinese Restaurant,Italian Restaurant,Bus Line,Dim Sum Restaurant,Bike Shop
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
7,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,1.0,Coffee Shop,Middle Eastern Restaurant,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Bridal Shop,Pharmacy,Pizza Place
8,M2K,North York,Bayview Village,43.786947,-79.385975,3.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop,Deli / Bodega
9,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,5.0,Airport,Other Repair Shop,Park,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop


Visualize clusters 

In [304]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
       color=rainbow[int(cluster)-1],
       fill=True,
       fill_color=rainbow[int(cluster)-1],
       fill_opacity=0.7).add_to(map_clusters
    )
       
map_clusters

<b> Now it is possibile to examine each cluster </b>

In [305]:
scar_merged.loc[scar_merged['Cluster Labels'] == 1, place_merged.columns[[1] + list(range(6, place_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Women's Store,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
2,North York,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Fraternity House,Vietnamese Restaurant,Accessories Store
5,North York,Coffee Shop,Gym,Asian Restaurant,Beer Store,Clothing Store,Chinese Restaurant,Italian Restaurant,Bus Line,Dim Sum Restaurant,Bike Shop
6,North York,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
7,North York,Coffee Shop,Middle Eastern Restaurant,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Bridal Shop,Pharmacy,Pizza Place
14,North York,Food Truck,Korean Restaurant,Home Service,Baseball Field,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
15,North York,Fast Food Restaurant,Italian Restaurant,Coffee Shop,Cupcake Shop,Japanese Restaurant,Juice Bar,Café,Liquor Store,Ice Cream Shop,Butcher
17,North York,Restaurant,Coffee Shop,Ramen Restaurant,Sushi Restaurant,Japanese Restaurant,Café,Sandwich Place,Pharmacy,Vietnamese Restaurant,Movie Theater
18,North York,Discount Store,Grocery Store,Liquor Store,Athletics & Sports,Gym / Fitness Center,Women's Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop


In [306]:
place_merged.loc[place_merged['Cluster Labels'] == 2, place_merged.columns[[1] + list(range(6, place_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,Baseball Field,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner


In [283]:
place_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(6, scar_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Baseball Field,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cupcake Shop
4,North York,Park,Japanese Restaurant,Pizza Place,Pub,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,North York,Chinese Restaurant,Café,Bank,Japanese Restaurant,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop,Deli / Bodega


In [284]:
scar_merged.loc[scar_merged['Cluster Labels'] == 4, scar_merged.columns[[1] + list(range(6, scar_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,North York,Cafeteria,Women's Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner


In [285]:
scar_merged.loc[scar_merged['Cluster Labels'] == 5, scar_merged.columns[[1] + list(range(6, scar_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
9,North York,Airport,Other Repair Shop,Park,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
11,North York,Basketball Court,Park,Construction & Landscaping,Bakery,Deli / Bodega,Dog Run,Comfort Food Restaurant,Convenience Store,Cupcake Shop,Dim Sum Restaurant
19,North York,Park,Convenience Store,Bank,Bar,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cupcake Shop,Deli / Bodega


In [287]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(6, scar_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,Pharmacy,Pizza Place,Empanada Restaurant,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
20,North York,Discount Store,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cupcake Shop
